In [3]:
from tkinter import *
from tkinter import messagebox
from tkinter import filedialog
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from PIL import ImageTk,Image
import cv2

def clean_text(msg):
    sp_words=stopwords.words('english')
    sp_words.remove('not')
    sp_words.remove("don't")
    sp_words.remove("didn't")
    sp_words.remove("hasn't")
    sp_words.remove("haven't")
    sp_words.remove("wasn't")
    sp_words.remove("weren't")

    def remove_punct(msg):
        return re.sub(f'[{string.punctuation}]','',msg)

    def remove_stopwds(msg):
        words=word_tokenize(msg)
        new_words=[]
        for w in words:
            if(w not in sp_words):
                new_words.append(w)
        return " ".join(new_words)

    def stemming(msg):
        ps=PorterStemmer()
        words=word_tokenize(msg)
        new_words=[]
        for w in words:
            new_words.append(ps.stem(w))
        return " ".join(new_words) 
    
    X1=remove_punct(msg)
    X2=X1.lower()
    X3=remove_stopwds(X2)
    X4=stemming(X3)
    return X4

df=pd.read_csv('C:/users/drishya/dataset/sentiment/Restaurant_Reviews.txt',delimiter="\t")
df.Review=list(map(clean_text,df.Review))
cv=CountVectorizer(binary=False,ngram_range=(1,2))
X=cv.fit_transform(df.Review).toarray()
y=df.Liked
clf=MultinomialNB()
clf.fit(X,y)

win=Tk()
win.state("zoomed")
win.resizable(width=False,height=False)
'''canvas=Canvas(win)
img=ImageTk.PhotoImage(Image.open('c:/users/drishya/Desktop/download.jpeg').resize((180,130)))
canvas.create_image(0,0,anchor=NW,image=img)
canvas.pack(expand=YES,fill=BOTH) '''      


win.configure(bg="black")
win.title("My project")

lbl_title=Label(win,text="Projects",font=('',50,'underline'),bg="black",fg="white")
lbl_title.place(relx=.40,rely=0.04)


def home_screen():
    frm=Frame(win,bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
    
    lbl_user=Label(frm,text="-By Drishya Uniyal",font=('',20,'bold'),bg='black',fg="white")
    lbl_user.place(relx=.8,rely=.1)
    
    lbl_user=Label(frm,text="Username",font=('',20,'bold'),bg='black',fg="white")
    lbl_user.place(relx=.3,rely=.2)

    entry_user=Entry(frm,font=('',20,'bold'),bd=10)
    entry_user.place(relx=.42,rely=.2)
    entry_user.focus()

    lbl_pass=Label(frm,text="Password",font=('',20,'bold'),bg='black',fg="white")
    lbl_pass.place(relx=.3,rely=.3)

    entry_pass=Entry(frm,font=('',20,'bold'),bd=10,show="*")
    entry_pass.place(relx=.42,rely=.3)

    btn_login=Button(frm,command=lambda:welcome_screen(entry_user,entry_pass),text="login",font=('',20,'bold'),bd=10,width=10)
    btn_login.place(relx=.46,rely=.4)
    
    
def welcome_screen(entry_user=None,entry_pass=None):
    if(entry_user!=None and entry_pass!=None):
        user=entry_user.get()
        pwd=entry_pass.get()
    else:
        user="admin"
        pwd="admin"
    if(len(user)==0 or len(pwd)==0):
        messagebox.showwarning("validation","Please fill both fields")
        return
    else:
        if(user=="admin" or pwd=="admin"):
            frm=Frame(win,bg='black')
            frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
            
            btn_single=Button(frm,command=lambda:welcome(),text="Review Analysis",font=('',20,'bold'),bd=10,width=25)
            btn_single.place(relx=.35,rely=.2)

            btn_bulk=Button(frm,command=lambda:spam_detection(),text="Spam Detection",font=('',20,'bold'),bd=10,width=25)
            btn_bulk.place(relx=.35,rely=.3)
            
            btn_bulk=Button(frm,command=lambda:welcome_screen_new(),text="Face Detection",font=('',20,'bold'),bd=10,width=25)
            btn_bulk.place(relx=.35,rely=.4)

            btn_logout=Button(frm,command=lambda:logout(),text="logout",font=('',20,'bold'),bd=10)
            btn_logout.place(relx=.9,rely=0)
        else:
            messagebox.showerror("Fail","Invalid Username/Password")
    
    
def predict_single(entry_user,lbl_result):
    user_review=entry_user.get()
    ct=clean_text(user_review)
    X_test=cv.transform([ct]).toarray()
    pred=clf.predict(X_test)
    if(pred[0]==0):
        lbl_result.configure(text="Not Liked",font=('',25,'italic'),fg="red")
    else:
        lbl_result.configure(text="Liked",font=('',25,'italic'),fg="white")

def predict_save(entry_src,entry_dest):        
    srcpath=entry_src.get()
    destpath=entry_dest.get()
    df=pd.read_csv(srcpath,names=['Review'])
    X=df.Review.map(clean_text)
    X_test=cv.transform(X).toarray()  
    pred=clf.predict(X_test)
    result_df=pd.DataFrame()
    result_df['Review']=df.Review
    result_df['Sentiment']=pred
    result_df['Sentiment']=result_df['Sentiment'].map({0:"Not Liked",1:"Liked"})
    result_df.to_csv(destpath,index=False,sep="\t")
    messagebox.showinfo('Result',"Prediction Done...")
    
    
def logout():
    option=messagebox.askyesno('Confirmation','Are you sure you want to logout?')
    if(option==True):
        home_screen()
    else:
        pass

            
            
def welcome():
    
    frm=Frame(win,bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
    
    
    lbl_title=Label(frm,text="Review Analysis",font=('',50,'underline'),bg='black',fg="white")
    lbl_title.place(relx=.33,rely=0.04)
            
    btn_single=Button(frm,command=lambda:single_feedback_screen(),text="Single Feedback Prediction",font=('',20,'bold'),bd=10,width=25)
    btn_single.place(relx=.35,rely=.2)

    btn_bulk=Button(frm,command=lambda:bulk_feedback_screen(),text="Bulk Feedback Prediction",font=('',20,'bold'),bd=10,width=25)
    btn_bulk.place(relx=.35,rely=.4)

    btn_logout=Button(frm,command=lambda:logout(),text="logout",font=('',20,'bold'),bd=10)
    btn_logout.place(relx=.9,rely=0)
    
    btn_back=Button(frm,command=lambda:welcome_screen(),text="back",font=('',20,'bold'),bd=10)
    btn_back.place(relx=0,rely=0)

    
    
def single_feedback_screen():
    frm=Frame(win,bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
    
    lbl_user=Label(frm,text="Enter Feedback:",font=('',20,'bold'),bg='black',fg="white")
    lbl_user.place(relx=.28,rely=.3)

    entry_user=Entry(frm,font=('',20,'bold'),bd=10)
    entry_user.place(relx=.45,rely=.3)
    entry_user.focus()

    lbl_result=Label(frm,text="Prediction:",font=('',20,'bold'),bg='black',fg="white")
    lbl_result.place(relx=.35,rely=.55)

    btn_login=Button(frm,command=lambda:predict_single(entry_user,lbl_result),text="predict",font=('',20,'bold'),bd=10,width=8)
    btn_login.place(relx=.47,rely=.42)

    btn_back=Button(frm,command=lambda:welcome_screen(),text="back",font=('',20,'bold'),bd=10)
    btn_back.place(relx=.9,rely=0)
    

def bulk_feedback_screen():
    frm=Frame(win,bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
    
    lbl_src=Label(frm,text="Select Source file:",font=('',20,'bold'),bg='black',fg="white")
    lbl_src.place(relx=.19,rely=.2)
    
    lbl_dest=Label(frm,text="Select Destination Directory:",font=('',20,'bold'),bg='black',fg="white")
    lbl_dest.place(relx=.19,rely=.32)
    

    entry_src=Entry(frm,font=('',20,'bold'),bd=10)
    entry_src.place(relx=.45,rely=.2)
    entry_src.focus()

    entry_dest=Entry(frm,font=('',20,'bold'),bd=10)
    entry_dest.place(relx=.45,rely=.32)
    
    btn_browse=Button(frm,command=lambda:browse(entry_src),text="browse",font=('',20,'bold'),bd=10,width=8)
    btn_browse.place(relx=.7,rely=.2)

    btn_browse2=Button(frm,command=lambda:browse2(entry_dest),text="browse",font=('',20,'bold'),bd=10,width=8)
    btn_browse2.place(relx=.7,rely=.32)
    
    btn_login=Button(frm,command=lambda:predict_save(entry_src,entry_dest),text="predict and save",font=('',20,'bold'),bd=10,width=15)
    btn_login.place(relx=.47,rely=.5)

    btn_back=Button(frm,command=lambda:welcome_screen(),text="back",font=('',20,'bold'),bd=10)
    btn_back.place(relx=.9,rely=0)
    
    
def browse(entry_path):
    file_path=filedialog.askopenfilename()
    entry_path.delete(0,END)
    entry_path.insert(0,file_path)

def browse2(entry_path):
    file_path=filedialog.askdirectory()+"/result.txt"
    entry_path.delete(0,END)
    entry_path.insert(0,file_path)   
    
''' ------------------------------------for spam detection code starts here-----------------------------'''
    
def spam_detection():
    df=pd.read_csv('C:/users/drishya/dataset/sentiment/spam_ham.txt',delimiter="\t",header=None,names=['msg_type','msg'])
    df.msg=list(map(clean_text,df.msg))
    cv=TfidfVectorizer(binary=False)
    X=cv.fit_transform(df.msg).toarray()
    y=df.msg_type
    smote=SMOTE()
    new_X,new_y=smote.fit_resample(X,y)
    clf=MultinomialNB()
    clf.fit(new_X,new_y)    
    frm=Frame(win,bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
    
    
    lbl_title=Label(frm,text="Spam Detection",font=('',50,'underline'),bg='black',fg="white")
    lbl_title.place(relx=.34,rely=0.04)
            
    btn_single=Button(frm,command=lambda:single_feedback_screen1(),text="Single Feedback Prediction",font=('',20,'bold'),bd=10,width=25)
    btn_single.place(relx=.35,rely=.2)

    btn_bulk=Button(frm,command=lambda:bulk_feedback_screen1(),text="Bulk Feedback Prediction",font=('',20,'bold'),bd=10,width=25)
    btn_bulk.place(relx=.35,rely=.4)

    btn_logout=Button(frm,command=lambda:logout(),text="logout",font=('',20,'bold'),bd=10)
    btn_logout.place(relx=.9,rely=0)
    
    btn_back=Button(frm,command=lambda:welcome_screen(),text="back",font=('',20,'bold'),bd=10)
    btn_back.place(relx=0,rely=0)
    
    
def predict_single1(entry_user,lbl_result):
    user_review=entry_user.get()
    ct=clean_text(user_review)
    X_test=cv.transform([ct]).toarray()
    pred=clf.predict(X_test)
    if(pred[0]=='ham'):
        lbl_result.configure(text="Ham",fg="white")
    else:
        lbl_result.configure(text="Spam",fg="red")

def predict_save1(entry_src,entry_dest):        
    srcpath=entry_src.get()
    destpath=entry_dest.get()
    df=pd.read_csv(srcpath,header=None,names=['msg'])
    X=df.msg.map(clean_text)
    X_test=cv.transform(X).toarray()  
    pred=clf.predict(X_test)
    result_df=pd.DataFrame()
    result_df['msg']=df.msg
    result_df['msg_type']=pred
    result_df['msg_type']=result_df['msg_type']
    result_df.to_csv(destpath,index=False,sep="\t")
    messagebox.showinfo('Result',"Prediction Done...")  
    
    
def single_feedback_screen1():
    frm=Frame(win,bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
    
    lbl_user=Label(frm,text="Enter Msg:",font=('',20,'bold'),bg='black',fg="white")
    lbl_user.place(relx=.28,rely=.3)

    entry_user=Entry(frm,font=('',20,'bold'),bd=10)
    entry_user.place(relx=.45,rely=.3)
    entry_user.focus()

    lbl_result=Label(frm,text="Prediction:",font=('',20,'bold'),bg='black',fg="white")
    lbl_result.place(relx=.35,rely=.55)

    btn_login=Button(frm,command=lambda:predict_single1(entry_user,lbl_result),text="predict",font=('',20,'bold'),bd=10,width=8)
    btn_login.place(relx=.47,rely=.42)

    btn_back=Button(frm,command=lambda:welcome_screen(),text="back",font=('',20,'bold'),bd=10)
    btn_back.place(relx=.9,rely=0)

def bulk_feedback_screen1():
    frm=Frame(win,bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
    
    lbl_src=Label(frm,text="Select Source file:",font=('',20,'bold'),bg='black',fg="white")
    lbl_src.place(relx=.26,rely=.2)
    
    lbl_dest=Label(frm,text="Select Destination Directory:",font=('',20,'bold'),bg='black',fg="white")
    lbl_dest.place(relx=.19,rely=.32)
    

    entry_src=Entry(frm,font=('',20,'bold'),bd=10)
    entry_src.place(relx=.45,rely=.2)
    entry_src.focus()

    entry_dest=Entry(frm,font=('',20,'bold'),bd=10)
    entry_dest.place(relx=.45,rely=.32)
    
    btn_browse=Button(frm,command=lambda:browse(entry_src),text="browse",font=('',20,'bold'),bd=10,width=8)
    btn_browse.place(relx=.7,rely=.2)

    btn_browse2=Button(frm,command=lambda:browse2(entry_dest),text="browse",font=('',20,'bold'),bd=10,width=8)
    btn_browse2.place(relx=.7,rely=.32)
    
    btn_login=Button(frm,command=lambda:predict_save1(entry_src,entry_dest),text="predict and save",font=('',20,'bold'),bd=10,width=15)
    btn_login.place(relx=.47,rely=.5)

    btn_back=Button(frm,command=lambda:welcome_screen(),text="back",font=('',20,'bold'),bd=10)
    btn_back.place(relx=.9,rely=0)
    
'''-----------------------------------code for face detection starts here----------------------------------------------------'''


def welcome_screen_new(entry_user=None,entry_pass=None):
    
    frm=Frame(win,bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)    

    lbl_title=Label(frm,text="Face Detection",font=('',50,'underline'),bg='black',fg="white")
    lbl_title.place(relx=.34,rely=0.04)
            
    btn_single=Button(frm,command=lambda:welcome2(),text="Using Image",font=('',20,'bold'),bd=10,width=25)
    btn_single.place(relx=.35,rely=.2)

    btn_bulk=Button(frm,command=lambda:welcome3(),text="Using Video",font=('',20,'bold'),bd=10,width=25)
    btn_bulk.place(relx=.35,rely=.3)
            
    btn_bulk=Button(frm,command=lambda:welcome4(),text="Using Webcam",font=('',20,'bold'),bd=10,width=25)
    btn_bulk.place(relx=.35,rely=.4)


    btn_logout=Button(frm,command=lambda:logout(),text="logout",font=('',20,'bold'),bd=10)
    btn_logout.place(relx=.9,rely=0)
    
            
    
def welcome2():
    frm=Frame(win,bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)    
    btn_back=Button(frm,command=lambda:welcome_screen(),text="back",font=('',20,'bold'),bd=10)
    btn_back.place(relx=.9,rely=0)
    
    Button(frm,command=lambda:browse(),text='browse image',font=('',20,'bold'),bd=10).place(relx=.4,rely=.1)
    Label(frm,font=('',40,'bold','underline'),bg='black',fg="white").pack()
    clf=cv2.CascadeClassifier('c:/users/drishya/dataset/haar/haarcascade_frontalface_default.xml')   
    
    
    def browse():
        global imageFrame   
        imageFrame=None
        if(imageFrame!=None):
            imageFrame.destroy()
        file_path=filedialog.askopenfilename()
        frame=cv2.imread(file_path)
        cv2image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)    
        imageFrame = Frame(frm,width=700, height=500,bd=10,bg='black')
        lmain = Label(imageFrame)
        lmain.grid(row=0, column=0)
        imageFrame.place(relx=.37,rely=.3)
        img = Image.fromarray(cv2image)
        imgtk = ImageTk.PhotoImage(image=img)
        lmain.imgtk = imgtk
        lmain.configure(image=imgtk)
        Button(win,command=lambda:startface(frame,cv2image,lmain),text='detect face',font=('',20,'bold'),bd=10,width=12).place(relx=.4,rely=.9)
    
    def startface(frame,cv2image,lmain):
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) 
        faces=clf.detectMultiScale(gray,1.2,5)
        for x,y,w,h in faces:
            cv2.rectangle(cv2image,(x,y),(x+w,y+h),(255,0,0),2)
        img = Image.fromarray(cv2image)
        imgtk = ImageTk.PhotoImage(image=img)
        lmain.imgtk = imgtk
        lmain.configure(image=imgtk)
        
def welcome4():
    frm=Frame(win,bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
    btn_StartCam=Button(command=lambda:start(),text='start Camera ',font=('',20,'bold'),bd=10).place(relx=.01,rely=.2)
    btn_StopCam=Button(command=lambda:stop(),text='stop camera',font=('',20,'bold'),bd=10).place(relx=.01,rely=.4)
    btn_back=Button(frm,command=lambda:welcome_screen(),text="back",font=('',20,'bold'),bd=10)
    btn_back.place(relx=.9,rely=0)    
    flag=True
    clf=cv2.CascadeClassifier('c:/users/drishya/dataset/haar/haarcascade_frontalface_default.xml')
    def stop():
        cap.release()
        lmain.destroy()
        imageFrame.destroy()

    def startfac():
        global flag
        flag=True

    def stopface():
        global flag
        flag=False

    def show_frame():
        _, frame = cap.read()
        cv2image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) 
        faces=clf.detectMultiScale(gray,1.3,7)
        for x,y,w,h in faces:
            if(flag==True):
                cv2.rectangle(cv2image,(x,y),(x+w,y+h),(255,0,0),2)
        img = Image.fromarray(cv2image)
        imgtk = ImageTk.PhotoImage(image=img)
        lmain.imgtk = imgtk
        lmain.configure(image=imgtk)
        lmain.after(10, show_frame) #calls show_frame after 10 mills

    def start():
        global lmain,cap,lmain,imageFrame
        cap=cv2.VideoCapture(0)
        imageFrame=Frame(width=1500,height=2000,bd=1,bg='green')
        lmain=Label(imageFrame)
        lmain.grid(row=5, column=10)
        imageFrame.place(relx=.3,rely=.3)
        btn_DetectFace=Button(imageFrame,command=lambda:startfac(),text='detect face',font=('',20,'bold'),bd=10,width=12).place(relx=.1,rely=.9)
        btn_Stop=Button(imageFrame,command=lambda:stopface(),text='stop detecting',font=('',20,'bold'),bd=10,width=12).place(relx=.5,rely=.9)

        show_frame()

    
    
def welcome3():
    frm=Frame(win,bg='black')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)    
    
    btn_back=Button(frm,command=lambda:welcome_screen(),text="back",font=('',20,'bold'),bd=10)
    btn_back.place(relx=.9,rely=0)
    
    Button(frm,command=lambda:browse(),text='browse video',font=('',20,'bold'),bd=10).place(relx=.4,rely=.1)
    Label(frm,font=('',40,'bold','underline'),bg='black',fg="white").pack()
    clf=cv2.CascadeClassifier('c:/users/dataset/haar/haarcascade_frontalface_default.xml')  
    
    def browse():
        global imageFrame   
        imageFrame=None
        if(imageFrame!=None):
            imageFrame.destroy()
        file_path=filedialog.askopenfilename()
        vdo=cv2.VideoCapture(file_path)
        while(True):
            flag,img=vdo.read()
            if(flag==True):
                gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                faces=clf.detectMultiScale(gray,1.4,4)
                for x,y,w,h in faces:
                    cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)
                cv2.imshow('vdo',img)
                k=cv2.waitKey(1)
                if(k==ord('q')):
                    break  
            else:
                break   
        vdo.release()
        cv2.destroyAllWindows()  
    

    
home_screen()    
win.mainloop() #to make window visible
